In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
 !pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 25.1 MB/s eta 0:00:00


In [10]:
!pip install gtts


In [12]:
!pip install playsound


  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=8d4e5108b93b6244a95936532364ef063a979ca2c55bcef8c14cac081ef44cd8
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [16]:
!apt-get install portaudio19-dev
!pip install pyaudio


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 31 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 2s (107 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121730 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [32]:
# Import libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import speech_recognition as sr
from gtts import gTTS
from playsound import playsound
import IPython.display as ipd
from IPython.display import Audio, display

# Download NLTK stopwords dataset
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:

# Load the dataset
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Corpus - QA.csv")


In [23]:
# Preprocess text for Jaccard Similarity
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

In [24]:
def get_most_similar_question(user_input, corpus):
    user_input = preprocess_text(user_input)
    corpus['Similarity'] = corpus['Processed_Text'].apply(lambda x: len(set(user_input.split()) & set(x.split())) / float(len(set(user_input.split()) | set(x.split()))))
    most_similar_index = corpus['Similarity'].idxmax()
    return corpus.loc[most_similar_index, 'Answer']


In [33]:
def text_to_speech(text):
    tts = gTTS(text=text, lang='en', slow=False)
    tts.save("output.mp3")
    display(Audio("output.mp3", autoplay=True))

In [26]:
# Updated Speech to Text function
def speech_to_text(audio_file_path):
    recognizer = sr.Recognizer()

    try:
        with sr.AudioFile(audio_file_path) as source:
            audio = recognizer.record(source)

        user_input = recognizer.recognize_google(audio)
        print(f"User Input: {user_input}")
        return user_input
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return None
    except sr.RequestError as e:
        print(f"Error with the speech recognition service: {e}")
        return None


In [27]:

# Preprocess the entire dataset for Jaccard Similarity
dataset['Processed_Text'] = dataset['Question'].apply(preprocess_text)

In [35]:


# Main Chatbot Loop
# while count:
audio_file_path = "/content/drive/MyDrive/Colab Notebooks/voice.wav"
user_input = speech_to_text(audio_file_path)

if user_input is not None:
    answer = get_most_similar_question(user_input, dataset)
    print(f"Chatbot: {answer}")
    text_to_speech(answer)


User Input: where is Canada located
Chatbot: Yes.


In [40]:
from transformers import pipeline
from IPython.display import Audio, display

# Load sentiment analysis model
sentiment_analysis = pipeline("sentiment-analysis")

# # Updated Text to Speech function
# def text_to_speech(text):
#     tts = gTTS(text=text, lang='en', slow=False)
#     tts.save("output.mp3")
#     display(Audio(filename="output.mp3", autoplay=True))

# Sentiment Analysis function
def analyze_sentiment(user_input):
    result = sentiment_analysis(user_input)
    return result[0]['label']

# Main Sentiment Analysis Loop
# while True:
audio_file_path = "/content/drive/MyDrive/Colab Notebooks/voice.wav"
user_input = speech_to_text(audio_file_path)

if user_input is not None:
    sentiment = analyze_sentiment(user_input)
    sentiment_message = "Your sentence is positive." if sentiment == "POSITIVE" else "Your sentence is negative."
    print(sentiment_message)
    text_to_speech(sentiment_message)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


User Input: where is Canada located
Your sentence is positive.


In [45]:
from transformers import MarianMTModel, MarianTokenizer

# Load the MarianMT model and tokenizer for English to Urdu translation
model_name = "Helsinki-NLP/opus-mt-en-ur"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def text_to_speech_Urdu(text, target_language='en'):
    tts = gTTS(text=text, lang=target_language, slow=False)
    tts.save("output.mp3")
    display(Audio(filename="output.mp3", autoplay=True))

# Translate function
def translate_text(text, model, tokenizer, target_language="ur"):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    translation_ids = model.generate(**inputs, max_length=100, num_beams=4, length_penalty=2.0, early_stopping=True)
    translated_text = tokenizer.decode(translation_ids[0], skip_special_tokens=True)
    return translated_text

# Main Translation Loop

    # Replace with your user input or speech-to-text result
user_input = sentiment_message

translated_text = translate_text(user_input, model, tokenizer)
print(f"Translated text (Urdu): {translated_text}")

text_to_speech_Urdu(translated_text, target_language='ur')


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Translated text (Urdu): آپ کی سزا مثبت ہے.
